In [ ]:
import copy
import numpy as np
import open3d as o3d
import random

def generate_random_colors(N):
    colors = [[0, 0, 0]]
    for _ in range(N):
        colors.append(
            [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
        )

    colors = np.vstack(colors) / 255
    return colors

def color_pcd_by_labels1(pcd, labels):
    colors = generate_random_colors(len(labels) + 1)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colored.colors = o3d.utility.Vector3dVector(
        np.zeros(np.asarray(pcd.points).shape)
    )

    for i in range(len(pcd_colored.points)):
        if labels[i] == 0 or labels[i] == -1:
            continue
        pcd_colored.colors[i] = colors[labels[i]]

    return pcd_colored

In [ ]:
import pickle

start_num = 20
end_num = start_num + 4

file_name = "experiment_bin_0704_4_sem_offset0_T0l03/start{}_end{}.pickle".format(start_num, end_num)

with open(file_name, 'rb') as file:
    data = pickle.load(file)

print(data[0])  # значения гиперпараметров запуска

In [ ]:
from sklearn.cluster import HDBSCAN

clusterer = HDBSCAN()

In [ ]:
import numpy as np
import open3d as o3d

X = np.asarray(data[1])  # точки обработанного облака ДО вокселизации
pcd_hdbscan = o3d.geometry.PointCloud()
pcd_hdbscan.points = o3d.utility.Vector3dVector(X)

In [ ]:
import copy

# проведение точно такой же вокселизации, как и в моем алгоритме

pcd_hdbscan_copy = copy.deepcopy(pcd_hdbscan)

min_bound = pcd_hdbscan_copy.get_min_bound()
max_bound = pcd_hdbscan_copy.get_max_bound()

downpcd_trace = pcd_hdbscan_copy.voxel_down_sample_and_trace(
    0.25, min_bound, max_bound, True
)

downpcd = downpcd_trace[0]
trace_hdbscan = downpcd_trace[2]

print("voxel_down_pcd_size={}".format(len(downpcd.points)))

In [ ]:
# проверяем, что вокселизация привела облако в то состояние,
# при котором проводилась сегментация моим алгоритмом

trace_arrays = []
for int_vector in trace_hdbscan:
    trace_arrays.append(np.asarray(int_vector))

for ind, el in enumerate(trace_arrays):
    if (el != data[2][ind]).any():
        print("ERROR")

In [ ]:
# сегментация HDBSCAN

clusters = clusterer.fit_predict(np.asarray(downpcd.points))
print(set(clusters))

In [ ]:
import copy

from src.utils.pcd_utils import visualize_pcd

colored_pcd = color_pcd_by_labels1(copy.deepcopy(downpcd), clusters)
visualize_pcd(colored_pcd)

In [ ]:
# приводим результаты кластеризации к листу, элементы которого - массивы точек одного кластера

clusters_list = []

print("before set(clusters)={}".format(set(clusters)))
# метки 0 и -1 означают шум, они для нас равны
for ind, label in enumerate(clusters):
    if label == -1:
        clusters[ind] = 0
print("after set(clusters)={}".format(set(clusters)))

# инициализация листа
for label in set(clusters):
    clusters_list.append([])

for ind, label in enumerate(clusters):
    clusters_list[label].append(ind)

In [ ]:
import copy

from src.utils.pcd_utils import color_pcd_by_clusters_and_voxels
from src.utils.pcd_utils import visualize_pcd

colored_pcd = color_pcd_by_clusters_and_voxels(
    copy.deepcopy(pcd_hdbscan_copy), copy.deepcopy(trace_hdbscan), clusters_list
)
visualize_pcd(colored_pcd)

In [ ]:
# выборка из кластеризации только инстансов

def find_num_in_inst_label_array(src_points, inst_label_array_for_clustering):
    for point in src_points:
        if inst_label_array_for_clustering[point] > 0:
            return inst_label_array_for_clustering[point]
    return -1


def build_pred_inst_array(
    inst_label_array_for_clustering, clusters, trace, instance_threshold
):
    pred_inst_array = np.zeros(len(inst_label_array_for_clustering), dtype=int)
    free_id = 1
    for cluster in clusters:
        voxel_not_in_gt_cluster_count = 0
        for voxel in cluster:
            src_points = trace[voxel]
            id = find_num_in_inst_label_array(
                src_points, inst_label_array_for_clustering
            )
            if id == -1:
                voxel_not_in_gt_cluster_count += 1

        cluster_in_gt_instance = (
            (len(cluster) - voxel_not_in_gt_cluster_count) / len(cluster)
        ) * 100
        if cluster_in_gt_instance >= instance_threshold:
            for voxel in cluster:
                src_points = trace[voxel]
                for src_point in src_points:
                    pred_inst_array[src_point] = free_id
            free_id += 1
    return pred_inst_array

In [ ]:
instance_threshold = 30
inst_label_array_for_clustering = data[4]  # из pickle

pred_inst_array = build_pred_inst_array(
    copy.deepcopy(inst_label_array_for_clustering),
    clusters_list,
    copy.deepcopy(trace_hdbscan),
    instance_threshold,
)

In [ ]:
from evops.metrics import precision
from evops.metrics import recall
from evops.metrics import fScore

pred_labels = pred_inst_array
gt_labels = inst_label_array_for_clustering
tp_condition = "iou"
print("precision={}".format(precision(pred_labels, gt_labels, tp_condition)))
print("recall={}".format(recall(pred_labels, gt_labels, tp_condition)))
print("fScore={}".format(fScore(pred_labels, gt_labels, tp_condition)))

In [ ]:
import copy

from src.utils.pcd_utils import color_pcd_by_labels
from src.utils.pcd_utils import visualize_pcd

colored_clusters_for_clustering = color_pcd_by_labels(
    copy.deepcopy(pcd_hdbscan_copy), pred_labels
)
visualize_pcd(colored_clusters_for_clustering)

In [ ]:
import copy

from src.utils.pcd_utils import color_pcd_by_labels
from src.utils.pcd_utils import visualize_pcd

colored_clusters_for_clustering = color_pcd_by_labels(
    copy.deepcopy(pcd_hdbscan_copy), gt_labels
)
visualize_pcd(colored_clusters_for_clustering)